In [1]:
import numpy as np
import scipy
import scipy.stats as stats
import matplotlib.pyplot as plt
import random as RD
from matplotlib import colors
import winsound
import csv
import copy

In [3]:
#It is important to leave numEquations and stepSize here, as they affect the integration.
numEquations = 4
stepSize = 0.1
simLength = 2000
tarray = np.arange(0,simLength,stepSize)
Ntimes = len(tarray)

spikeThreshold = 5 #Sets the voltage(mV) at which a spike is recorded. 

numnrn = 10 #Number of neurons in the model.

numSST = 0 #Number of SST neurons to be forced into the model.
percent_SST = 0 #Percent of neurons to be randomly assigned as SST interneurons.

c_e, c_i = 0.1, 0.5 #Percent connectivity. Excitory, Inhibitory.  
p_e, p_i = 0.8, 0 #Probabilty of an existing connection breaking and forming a new connection. Excitory, Inhibitory. 
local_conn = True #When true, new connections can be formed with local connections. When false, only non-local new
# connections are formed.

RD_seed = True # When true, a seed is used to generate connections
seed = 1 # The seed for generating random numbers/list indices. NOTE: defining a seed before a sequence of random events will
# not only define the outcome of the first random choice/event, but ALSO the following ones. So we only need one seed.

In [ ]:
class neuron:
        
    def __init__(self):
        self.ID = 0
        self.position = []
        self.connections = [] #List of (1 or 0) connection strengths to other neurons. Is tuple like [[postsyn,conn],[postsyn,conn]...].
        # DO NOT CHANGE self.connections from values of only 1 and 0 because many apects of the program rely on it.
        self.connectionWeights = [] #Holds changes made from plasticity. For self as presynaptic nrn. Values are strengths of
        # signal to other neurons from this neuron.
        self.Input_syn = 0
        self.Input_noise = 0
        self.Input_external = 0
        self.spikeTimes = [] # Set to record a spike when membrane voltage breaches variable spikeThreshold.
        self.prevActivity = 0
        self.neuronsInRange = [] #Tracks the # of neurons in range so as to minimize looping time during connection growth function
        self.solutions = np.zeros(numEquations) #Why does nrn.solutions still function as a comment?
        
        #Things I have added in myself:
        self.spike = False #Determines whether the neuron has already spiked or not. 
        self.Idrive = 0
        self.color = '' #Color of neuron for graphing.
        self.conn_in = [] #Connections coming in from other neurons. Sum is the in-degree of the neuron. Note, not tuple like self.connections.
        # Format is 1D list of connection strengths, where list index is presynaptic neuron. 
        self.category = 'Excitatory' #Labels the neuron type. Default is excitatory, can be chanegd to inhibitory. 
        self.gsyn = 1 # Connection strength multiplier.
        self.pair_spiketimes = np.zeros(numnrn) #Pair spike times for outgoing connections. Note that this only holds the most recent pair spiketime for each conn.
        self.start_noise = 0 #Starting step time for noise when it occurs (mV/stepSize). 
        self.backbone_ID = 0 # backbone_ID=0 will be used to designate lower E neurons and -1 for inhibitory neurons.
        self.spike_gaussian = [] #List of gaussian curves, each centered at the time a neuron spikes. Each index in list corresponds to a t_ind time.
        self.plas_on = True #Boolean determining whether or not to change plasticity of connections TO and FROM this nrn. 
        self.cw_in_history = [] #Connection weight history. Holds plasticity connection weights coming IN (this nrn as postsyn).
        # Set up as [[[weight from nrn 0, weight from nrn 1, ... ],time(ms)] ,...], one weight list for each milisecond.
        # List set up to skip first 500 ms because we don't want plasticity due to transient behaviors. Note the default
        # value for connections and non-existent connections is 1. 
        self.scatter_color = 'grey'
        self.scatter_quad_color = 'grey'
        self.i_current_hist = np.array([[0,time] for time in range(simLength)]) # Net inhibitory current to this neuron
        # every ms. [ [net current, time(ms)] , ...]
        
        
        
def equations(solns_, eqn, Isyn, Idrive,nrn):
    # These equations are parameters are adjusted to fit fast-spiking interneurons.
    
    tempVal = 0
    
    category = nrn.category
    Inoise = nrn.Input_noise #Noise from neuron. Maybe I should put Idrive, solns, etc here as well?
    
    if category == 'Excitatory':
        gks = gks_E
        C = 1 
        gna = 24
        gkdr = 3
        gl = 0.02
        Vna = 55
        Vk = -90
        Vl = -60
        if(eqn == 0):
            hinf = 1/(1+np.exp((solns_[3]+53)/7))
            tauh = .37 + 2.78/(1+np.exp((solns_[3]+40.5)/6))
            tempVal = (hinf - solns_[0])/tauh 
        elif(eqn == 1):
            ninf = 1/(1+np.exp((-solns_[3]-30)/10))
            taun = .37 + 1.85/(1+np.exp((solns_[3]+27)/15))
            tempVal = (ninf - solns_[1])/taun
        elif(eqn == 2):
            zinf = 1/(1+np.exp((-solns_[3]-39)/5))
            tempVal = (zinf - solns_[2])/75
        elif(eqn == 3):
            m = 1/(1+np.exp((-solns_[3]-30)/9.5))
            tempVal = (-gna*(m**3)*solns_[0]*(solns_[3]-Vna) - gkdr*(solns_[1]**4)*(solns_[3]-Vk) 
                       - gks*solns_[2]*(solns_[3]-Vk) - gl*(solns_[3]-Vl) + Idrive - Isyn + Inoise)/C
            
    elif category == 'SST':
        gks = gks_SST
        C = 1 
        gna = 24
        gkdr = 3
        gl = 0.02
        Vna = 55
        Vk = -90
        Vl = -60
        if(eqn == 0):
            hinf = 1/(1+np.exp((solns_[3]+53)/7))
            tauh = .37 + 2.78/(1+np.exp((solns_[3]+40.5)/6))
            tempVal = (hinf - solns_[0])/tauh 
        elif(eqn == 1):
            ninf = 1/(1+np.exp((-solns_[3]-30)/10))
            taun = .37 + 1.85/(1+np.exp((solns_[3]+27)/15))
            tempVal = (ninf - solns_[1])/taun
        elif(eqn == 2):
            zinf = 1/(1+np.exp((-solns_[3]-39)/5))
            tempVal = (zinf - solns_[2])/75
        elif(eqn == 3):
            m = 1/(1+np.exp((-solns_[3]-30)/9.5))
            tempVal = (-gna*(m**3)*solns_[0]*(solns_[3]-Vna) - gkdr*(solns_[1]**4)*(solns_[3]-Vk) 
                       - gks*solns_[2]*(solns_[3]-Vk) - gl*(solns_[3]-Vl) + Idrive - Isyn)/C
            
    elif category == 'PV+':
        C = 1 
        gna = 35
        gkdr = 9
        gl = 0.1
        Vna = 55
        Vk = -90
        Vl = -65    
        if(eqn == 0):
            a_h = 0.07*np.exp(-(solns_[3]+58)/20)
            b_h = 1/(np.exp(-0.1*(solns_[3]+28))+1)
            phi = 5
            tempVal = phi*(a_h*(1-solns_[0]) - b_h*solns_[0])
        elif(eqn == 1):
            a_n = -0.01*(solns_[3]+34)/(np.exp(-0.1*(solns_[3]+34))-1)
            b_n = 0.125*np.exp(-(solns_[3]+44)/80)
            phi = 5
            tempVal = phi*(a_n*(1-solns_[1])-b_n*solns_[1])
        elif(eqn == 2):
            zinf = 1/(1+np.exp((-solns_[3]-39)/5))
            tempVal = (zinf - solns_[2])/75
        elif(eqn == 3):
            a_m = -0.1*(solns_[3]+35)/(np.exp(-0.1*(solns_[3]+35))-1)
            b_m = 4*np.exp(-(solns_[3]+60)/18)
            m = a_m/(a_m+b_m)
            tempVal = (-gna*(m**3)*solns_[0]*(solns_[3]-Vna) - gkdr*(solns_[1]**4)*(solns_[3]-Vk) 
                       - gks*solns_[2]*(solns_[3]-Vk) - gl*(solns_[3]-Vl) + Idrive - Isyn)/C
        
    return tempVal





def RK4(t_ind):
    
    global neuron
    
    for nrn in neurons:
        
        
        solns = nrn.solutions
        Isyn = nrn.Input_syn
        Idrive = nrn.Idrive
        k1 = np.zeros(numEquations)
        k2 = np.zeros(numEquations)
        k3 = np.zeros(numEquations)
        k4 = np.zeros(numEquations)
        
        init_solns = solns
        
        #Calculates the k1 variables
        for ii in range(len(solns)):
            k1[ii] = stepSize*equations(solns, ii,Isyn,Idrive,nrn)

        #Calculates the k2 variables
        for ii in range(len(solns)):
            k2[ii] = stepSize*equations(solns+k1/2, ii,Isyn,Idrive,nrn) #important fix done here. solns must be advanced by k
                                                                    #for calculation of the next k variable.
        #Calculates the k3 variables
        for ii in range(len(solns)):
            k3[ii] = stepSize*equations(solns+k2/2, ii,Isyn,Idrive,nrn) 

        #Calculates the k4 variables
        for ii in range(len(solns)):
            k4[ii] = stepSize*equations(solns+k3, ii,Isyn,Idrive,nrn)
        
        #Updates the general solution
        for ii in range(len(solns)):
            solns[ii] = init_solns[ii] + (k1[ii] + 2*k2[ii] + 2*k3[ii] + k4[ii])/6 
            nrn.solutions[ii] = solns[ii]
            
            
            
def init_nrn(numnrn): #initializes neurons and assigns ID, connections, weights, etc. 
    global neuron
    neurons = [] #List containing neuron objects
    nconn_Mat = [np.empty(3)] # 2D matrix for storing new connections.
    
    if RD_seed: # When true, the simulation will be reproducable entirely (all connections, neuron assignments, initial coniditions).
        RD.seed(seed)
    
    def count_SST(neurons): # A function for counting the number of SST neurons.
        count = 0
        for nrn in neurons:
            if nrn.category == 'SST':
                count += 1
        return count

    
    for i in range(numnrn):  
        neurons = np.append(neurons,neuron()) #Intiallizes numnrn number of neurons
        
        
    #This for loop ensures that exactly numSST number of E neurons are changed to SST.
    for i in range(numSST):
        changed_to_SST = False #Keeps loop running until excitatory neuron is found to change to SST neuron.
        while changed_to_SST == False: #Loop mentioned above.
            nrn = RD.choice(neurons) #grabs one neuron object at random (available for editing)
            if nrn.category == 'Excitatory': #If true, turns excitatory neuron to SST. If neuron is not Excitatory, while loop runs again.
                nrn.category = 'SST'
                nrn.backbone_ID = -1 #Assigns inhibitory neurons to backbone ID = -1.
                changed_to_SST = True
                
    #Create list of only E neurons.
    Eneurons = []
    for nrn in neurons:
        if nrn.category == 'Excitatory':
            Eneurons.append(nrn) #Note that even though this is a different list than neurons, the neuron objects within can be
            # changed all the same like they were in neurons. 
                
                
    #Changes all excitatory neurons to having high inhibition (i.e. like LE neurons):
    for nrn in Eneurons:
        nrn.gsyn = gsynH
    
    
    
    def create_backbones(Eneurons):
        #this function initializes backbones into a network assuming that all E neurons have a high gysn (high inhibition level). 
        # Takes list of excitatory neuron objects as input. WILL NEED TO FIX WITH try->except WHEN num_per_bb > non_bb_size because
        #then LElist will sample too many elements from templist on last run of loop. 
        global neuron 

        num_bb = 2 #number of backbones to create from number of available E neurons
        non_bb_size = 40 #number of non-backbone E neurons to be left in network.
        num_bb_nrns = numnrn - (numSST + non_bb_size) #number of neurons for splitting into backbones.
        num_per_bb = int(num_bb_nrns/num_bb) # number of E neurons per backbone.

        LElist = RD.sample(Eneurons,num_per_bb) #temp list for looping

        for bb in range(1,num_bb+1): #This makes the notation easier by shifitng indicies +1. This is because nrn.backbone_ID=0
        # is reserved for NON-backbone neurons. bb=1 designates backbone_ID=1.
            templist=[]

            for nrn in LElist:
                nrn.backbone_ID = bb # Assigns first randomly chosen group of neurons to the first backbone.
                nrn.gsyn = gsynL #Backbones are created in the "on" state.
                
            for nrn in Eneurons: #Makes a new list with only non-backbone nrns.
                if nrn.backbone_ID == 0:
                    templist.append(nrn)
            LElist = RD.sample(templist,num_per_bb)

        
        
    create_backbones(Eneurons)


    ID = 0
    bb_colors = ['cyan','blue','green','orange','purple'] # Colors for backbones, cyan reserved for non-backbone E neurons.
    for nrn in neurons: #assigns neurons in list their IDs, init voltage, Idrive, etc.
        nrn.ID = ID
        ID += 1 
        nrn.spikeTimes = []
        nrn.solutions = [RD.random(),RD.random(),RD.random(),RD.uniform(-55,-20)] #Initial conditions of each neuron. Initial voltage randomly assigned between -55 and -20 mV.
        #nrn.gsyn = round(RD.uniform(0,3),4) #Chooses a connection multiplier between 0 and x.
        nrn.connectionWeights = [1]*numnrn #Creates a list of all connection weights to other neurons at value 1. 

        if nrn.category == 'Excitatory':
            nrn.Idrive = round(RD.uniform(Idrive_min, Idrive_max),3) #Random value between min and max rounded to 1 decimal places
            nrn.color = bb_colors[nrn.backbone_ID] #Assigns nrn color coded for backbone.
            
        if nrn.category == 'SST':
            nrn.color = 'Red' #Inhibitory given red.
            nrn.Idrive = Idrive_SST #Idrive for inhibitory neurons. 
        if nrn.category == 'PV+':
            nrn.color = 'darkorange'
            nrn.Idrive = round(RD.uniform(Idrive_PVplus_min,Idrive_PVplus_max),3)
    
    conn_Matrix = np.zeros((numnrn,numnrn)) #initializes matrix of zeros with numnrn x numnrn size. Row = nrn #, Column = connected nrn #
    # Fills matrix with connectivity based on proximity. conn_span # of neurons to right and left are given full connection. 
    for row_index, row in enumerate(conn_Matrix):
        for column_index, conn in enumerate(row):
            
            if neurons[row_index].category == 'Excitatory': #Determines which connectivity percent to use based on neuron category.
                conn_span = int(c_e*numnrn/2) #number of neurons to be connected on either side of a neuron.

                #sets neurons at +- conn_span from diagonal to full connectivity.
                if column_index >= row_index-conn_span and column_index <= row_index+conn_span:
                    conn = 1 
                #Full connectivity at edge case of first neurons connected to last neurons in ring.
                elif row_index-conn_span < 0 and column_index >= numnrn+row_index-conn_span:
                    conn = 1
                #Full connectivity at edge case of last neurons connected to first neurons in ring.
                elif row_index+conn_span > (numnrn-1) and column_index <= row_index-numnrn+conn_span:
                    conn = 1 
                #All other neurons have zero connectivity.
                else:
                    conn = 0
                # Sets diagonal entries to zero.
                if column_index == row_index:
                    conn = 0
                    
            elif neurons[row_index].category == 'SST' or neurons[row_index].category == 'PV+': # If the presynaptic neuron is inhibitory.
                if RD.random() <= c_i and column_index != row_index: # if a random between 0 and 1 is less than the connectivity percent. 
                    conn = 1
                else:
                    conn = 0
                    
            row[column_index] = conn  #Assigns the local connections.
        conn_Matrix[row_index] = row

    # Changes connections based on proability p. 
    for row_index, row in enumerate(conn_Matrix): 
        row_temp = row.copy() #used to store changes while deleting connections from new_conn_list. VERY IMPORTANT TO USE .copy()
         # otherwise row will change when row_temp is changed. This is how assignment works. 
        if neurons[row_index].category == 'Excitatory': #Determines which connectivity percent to use based on neuron category.
            conn_span = int(c_e*numnrn/2) #number of neurons to be connected on either side of a neuron.
            p = p_e
        elif neurons[row_index].category == 'SST' or neurons[row_index].category == 'PV+':
            conn_span = int(c_i*numnrn/2)
            p = p_i 

        for column_index, conn in enumerate(row):
            
            if conn != 0: #only for existing connections.
                if RD.random() <= p: # RD.random() selects random float between 0 and 1.

                    if local_conn == True: # Allows new local connections.
                        new_conn_list = np.append(np.arange(0,row_index,1),np.arange(row_index+1,numnrn,1)) #Creates list of
                        #all nrn IDs besides self.
                    if local_conn == False: #No new local connections.
                        #List of all nrns except local and self. Very gross and uses heaviside functions. May be simplifiable. 
                        new_conn_list = np.append(np.arange(numnrn - numnrn*np.heaviside(row_index-conn_span-1, 1)
                                    +(row_index+conn_span-numnrn+1)*np.heaviside(row_index+conn_span-numnrn,1),row_index-conn_span,1),
                                    np.arange(row_index+conn_span+1,(numnrn+row_index-conn_span)-
                                    (row_index-conn_span)*np.heaviside(row_index-conn_span, 1),1))
                     
                    for index, val in enumerate(row_temp):#Deletes established conns from new_conn_list, preventing double connections.

                        if val != 0: #Sorts out only established conns.
                            delindex = np.where(new_conn_list == index) #Finds where est. conn lies in new_conn_list.
                            if len(delindex[0]) > 0: #Stops error from having nothing to delete when local_conn = False. 
                                delindex = delindex[0][0] #grabs useful integer.
                                new_conn_list = np.delete(new_conn_list, delindex) #deletes from possible conns. 
    
                    nconn = RD.choice(new_conn_list) #Randomly selects one neuron to connect to. 
                    nconn_info = [[row_index, column_index, nconn]] # [neuron #, old connection, new connection]. Must be 2D.
                    nconn_Mat = np.concatenate((nconn_Mat,nconn_info)) #Adds this info to a matrix for later use.
                    
                    #Updates values of the array used in determining new connections. 
                    row_temp[int(column_index)] = 0 
                    row_temp[int(nconn)] = 1


    nconn_Mat = np.delete(nconn_Mat,0,0) #Removes np.empty dummy row from matrix.
    
    #Apply new connection changes.
    for info in nconn_Mat:
        conn_Matrix[int(info[0]),int(info[1])] = 0 #Sets old connection to zero.
        conn_Matrix[int(info[0]),int(info[2])] += 1 #Establishes connection or adds another connection.

    nc_Matrix = np.empty((numnrn,numnrn,2)) #Empty matrix to hold final values. nc means neuron # and connection strength. 
    count = 0
    # Creates 3D array, nc_matrix, storing (nrn #, conn strength to nrn receiving Isyn)
    for row in conn_Matrix:
        conn_tuple = list(enumerate(row)) #list of tuples with info (postsyn nrn #, recieving nrn conn strength)
        nc_Matrix[count] = conn_tuple 
        count += 1

    
    #Assigns neuron objects the list of tuple connections. 
    for nrn in neurons:
        nrn.connections = nc_Matrix[nrn.ID] #Outgoing connections for nrn.
        nrn.conn_in = nc_Matrix[:,nrn.ID][:,1]# Incoming connections for nrn. [0,0,1,1] would mean this neuron recieves no
        # signal from neurons 0 and 1, and full signal from neurons 2 and 3. 
    
    return neurons,nc_Matrix




    
            
def updateSyn(t_ind): #Gives synaptic input to all neurons on connection list
    #Includes changes in synaptic strengths. t_start is the time at which the presynaptic neuron's voltage breaches -20 mV.
    # Has been changed to normalize strength of inputs to a neuron by number of inputs. I.e sum of all inputs comes to w_max. 
    t_temp = 0 
    global neuron

    # AMPA connections
    w_EE = 0.15
    w_EI = 0.08
    # GABA A connections
    w_II = 0.15
    w_IE = 0
    # GABA B connections
    w_II_B = 0
    w_IE_B = 0.05
    
    tau = 0.5 #Time constant for fast-acting receptors.
    tau_B = 50 # Time constant for GABA B receptors, slow-acting.
    
    for nrn in neurons:# presynaptic neurons.         
        if len(nrn.spikeTimes) > 0: # To prevent errors of calling [-1] from an array without any entries. Can change to be l > 2, 3 ...
            t_temp = nrn.spikeTimes[-1] #grabs time this neuron spikes at.

            for conn in nrn.connections: #Gives all postsynaptic neurons Isyn corrspondping to their voltage.
                V = neurons[int(conn[0])].solutions[3] #Voltage of postsynaptic neuron. Note conn[1] is the connection strength and conn[0] is the ID.
                Isyn = 0 
                
                if nrn.category == 'SST' or nrn.category == 'PV+': # Handles GABA A and B receptors in postsyn  neurons.
                    E_syn = -75 #Chloride reversal potential. 
                    if neurons[int(conn[0])].category == 'SST' or neurons[int(conn[0])].category == 'PV+': # For I-I connections.
                        for w,t in (w_II,tau),(w_II_B,tau_B): #Sends two signals, one with w_II/tau and one with w_II_B/tau_B. 
                            Isyn += conn[1]*(w)*np.exp(-stepSize*(t_ind-t_temp)/t)*(V - E_syn) # t is tau here. 
                    if neurons[int(conn[0])].category == 'Excitatory': # For I->E connections.
                        gsyn = neurons[int(conn[0])].gsyn #Connection strength multiplier of postsynaptic neuron.
                        for w,t in (w_IE,tau),(w_IE_B,tau_B):
                            Isyn += gsyn*conn[1]*(w)*np.exp(-stepSize*(t_ind-t_temp)/t)*(V - E_syn) # t = tau again.
                        
                        # Recording inhibitory input to each LE neuron:
                        if neurons[int(conn[0])].backbone_ID == 0 and t_ind % (1/stepSize) == 0: 
                            gsyn = neurons[int(conn[0])].gsyn #Connection strength multiplier of postsynaptic neuron.
                            for w,t in (w_IE,tau),(w_IE_B,tau_B):
                                neurons[int(conn[0])].i_current_hist[int(t_ind*stepSize)][0] += gsyn*conn[1]*(w)*np.exp(-stepSize*(t_ind-t_temp)/t)*(V - E_syn) # t = tau again.
                                #print(nrn.i_current_hist[int(t_ind*stepSize)][0], 'time ', t_ind*stepSize)
                        
                            
                if nrn.category == 'Excitatory':
                    E_syn = 0 # Sodium reversal potential. E_syn = 0 for excitory synapse and E_syn = -75 mV for inhibitory synapse
                    if neurons[int(conn[0])].category == 'SST' or neurons[int(conn[0])].category == 'PV+': # For E->I connections.
                        Isyn = conn[1]*(w_EI)*np.exp(-stepSize*(t_ind-t_temp)/tau)*(V - E_syn)
                    if neurons[int(conn[0])].category == 'Excitatory': # For E-E connections.
                        Isyn = nrn.connectionWeights[int(conn[0])]*conn[1]*(w_EE)*np.exp(-stepSize*(t_ind-t_temp)/tau)*(V - E_syn)
                    
                neurons[int(conn[0])].Input_syn += Isyn #Isyn going to Postsynaptic neuron.


    
    
    
            
def updateSpikeTime(t_ind):
    global neuron
    # Variables for synaptic plasticity:
    A = 0.6/10 # The maximum amount a synapse can change in strength per spike.
    tau = 10 # Time constant for exponential function in ms.
    
    skip_time = 20 #Time before spikes start being recorded (in ms).
    if plas_skip:
        skip_time = 500 #skips first 500 ms to avoid making plasticity changes based on transient behaviors. Most useful for 
        # networks with high gks_Inhibitory levels. 
    
    for nrn in neurons:
        
        # Recording the incoming connection plasticity weights for each neuron:
        if t_ind % (1/stepSize) == 0 and t_ind*stepSize > skip_time: #Only runs code every ms (t_ind with only zero in decimal place). Also skips first 500 ms of no plasticity.
            presyn_weight_list = np.ones(numnrn) #List for storing weight connections from each presyn neuron.
            for presyn_ID in range(numnrn): #Loops through presynaptic connections to nrn.
                presyn_weight_list[int(presyn_ID)] = neurons[int(presyn_ID)].connectionWeights[int(nrn.ID)] #Records value 
                # of connection weight from presynaptic neuron.
            nrn.cw_in_history.append([presyn_weight_list,t_ind*stepSize]) #Appends conn weight values to history. 

        # Recording spike times:
        if nrn.solutions[3] >= spikeThreshold and nrn.spike == False and (t_ind*stepSize) > skip_time: #Selects spikes, skips anything before the first "skip_time" ms. 
            nrn.spikeTimes = np.append(nrn.spikeTimes, t_ind) #Records (time/stepSize) of a spike.
            nrn.spike = True
                        
            if nrn.plas_on == True: #If nrn's plas_on is True, plasticity(to and from) nrn is allowed to change. If False, it is frozen.

                #Changes synaptic weights. Note, nrn.connections is a tuple [postsyn, conn] while nrn.conn_in is simply a list of conns where the index is the presyn.
                # This works using only spikes that have already occured, so in-conns will always be strengthened and out-conns always weakened.
                for postsyn,conn in nrn.connections: # The outgoing connections. This weakens synapses. 

                    if conn == 1 and nrn.category == 'Excitatory' and neurons[int(postsyn)].category == 'Excitatory' \
                    and len(neurons[int(postsyn)].spikeTimes) > 0 and neurons[int(postsyn)].spikeTimes[-1] >= (nrn.spikeTimes[-1] - 30/stepSize)\
                    and neurons[int(postsyn)].plas_on == True: #Excludes all neurons that haven't spiked in the last 30 ms.
                            pair_spiketime = stepSize*(nrn.spikeTimes[-1] - (nrn.spikeTimes[-1]-neurons[int(postsyn)].spikeTimes[-1])/2) #Spike time of neuron pair in ms.

                            # If pair firing frequency is below dep_thr Hz. Also prevents initial pair_spiketimes value of zero from causing unwanted initial depotentiation.
                            if 1000/plas_thr > (pair_spiketime - nrn.pair_spiketimes[int(postsyn)]) >= 1000/dep_thr and nrn.pair_spiketimes[int(postsyn)] != 0:
                                nrn.connectionWeights[int(postsyn)] += -w_dep*A*np.exp((-stepSize*abs(neurons[int(postsyn)].spikeTimes[-1] 
                                                                                           - nrn.spikeTimes[-1]))/tau)# Weaken synapse.
                                if nrn.connectionWeights[int(postsyn)] < 0:
                                    nrn.connectionWeights[int(postsyn)] = 0 #This prevents synaptic weakening below zero, which would simulate inhibition.

                            if 1000/dep_thr > (pair_spiketime - nrn.pair_spiketimes[int(postsyn)]) >= 1000/pot_thr: #Determines pair firing frequency bsed off last fire. Selects firing between dep_thr and pot_thr frequencies (Hz).
                                nrn.connectionWeights[int(postsyn)] += -w_bi*A*np.exp((-stepSize*abs(neurons[int(postsyn)].spikeTimes[-1]
                                                                                                - nrn.spikeTimes[-1]))/tau)# Weaken synapse.
                                if nrn.connectionWeights[int(postsyn)] < 0:
                                    nrn.connectionWeights[int(postsyn)] = 0 #This prevents synaptic weakening below zero, which would simulate inhibition.
                                #print('Synapse from ',nrn.ID,' to ',postsyn,' weakened to ',nrn.connectionWeights[int(postsyn)])
                            if pair_spiketime - nrn.pair_spiketimes[int(postsyn)] < 1000/pot_thr: #If frequency is greater than pot_thr Hz. 
                                nrn.connectionWeights[int(postsyn)] += w_pot*A*np.exp((-stepSize*abs(neurons[int(postsyn)].spikeTimes[-1] - nrn.spikeTimes[-1]))/tau)# Strengthen synapse.
                                if nrn.connectionWeights[int(postsyn)] > w_max:
                                    nrn.connectionWeights[int(postsyn)] = w_max #Caps strength at w_max.

                            nrn.pair_spiketimes[int(postsyn)] = pair_spiketime #Updates pair spiketime.

                for presyn,conn in enumerate(nrn.conn_in): # Incoming connections.

                    if conn == 1 and nrn.category == 'Excitatory' and neurons[int(presyn)].category == 'Excitatory' \
                    and len(neurons[int(presyn)].spikeTimes) > 0 and neurons[int(presyn)].spikeTimes[-1] >= (nrn.spikeTimes[-1] - 30/stepSize)\
                    and neurons[int(presyn)].plas_on == True:
                        pair_spiketime = stepSize*(nrn.spikeTimes[-1] - (nrn.spikeTimes[-1]-neurons[int(presyn)].spikeTimes[-1])/2) #Spike time of neuron pair in ms.

                        # If pair firing frequency is below dep_thr Hz. Also prevents initial pair_spiketimes value of zero from causing unwanted initial depotentiation.
                        if 1000/plas_thr > (pair_spiketime - neurons[int(presyn)].pair_spiketimes[int(nrn.ID)]) >= 1000/dep_thr and neurons[int(presyn)].pair_spiketimes[int(nrn.ID)] != 0:  
                            neurons[int(presyn)].connectionWeights[int(nrn.ID)] += -w_dep*A*np.exp((-stepSize*abs(neurons[int(presyn)].spikeTimes[-1] 
                                                                                            - nrn.spikeTimes[-1]))/tau)# Weaken synapse.
                            if neurons[int(presyn)].connectionWeights[int(nrn.ID)] < 0:
                                neurons[int(presyn)].connectionWeights[int(nrn.ID)] = 0 #This prevents synaptic weakening below zero, which would simulate inhibition.

                        if 1000/dep_thr > (pair_spiketime - neurons[int(presyn)].pair_spiketimes[int(nrn.ID)]) >= 1000/pot_thr: 
                            neurons[int(presyn)].connectionWeights[int(nrn.ID)] += w_bi*A*np.exp((-stepSize*abs(neurons[int(presyn)].spikeTimes[-1] 
                                                                                            - nrn.spikeTimes[-1]))/tau) #Strengthens synapse.
                            if neurons[int(presyn)].connectionWeights[int(nrn.ID)] > w_max:
                                neurons[int(presyn)].connectionWeights[int(nrn.ID)] = w_max # Caps strength at w_max.
                            #print('Synapse from ',presyn,' to ',nrn.ID,' strengthened to ',neurons[int(presyn)].connectionWeights[int(nrn.ID)])
                        if pair_spiketime - neurons[int(presyn)].pair_spiketimes[int(nrn.ID)] < 1000/pot_thr: #For frequencies greater than pot_thr Hz.
                            neurons[int(presyn)].connectionWeights[int(nrn.ID)] += w_pot*A*np.exp((-stepSize*abs(neurons[int(presyn)].spikeTimes[-1] 
                                                                                            - nrn.spikeTimes[-1]))/tau) #Strengthens synapse.
                            if neurons[int(presyn)].connectionWeights[int(nrn.ID)] > w_max:
                                neurons[int(presyn)].connectionWeights[int(nrn.ID)] = w_max # Caps strength at w_max.

                        neurons[int(presyn)].pair_spiketimes[int(nrn.ID)] = pair_spiketime #Updates pair spiketime.   
                    
        if nrn.solutions[3] <= -30 and nrn.spike == True: #Resets the spiking status, allows for next spike to be recorded. 
            nrn.spike = False

In [ ]:
neurons,nc_Matrix = init_nrn(numnrn) #initializes neurons and creates universal list.
def mainProgramLoop():
    
    for t_ind in range(Ntimes):
        
        #Records timing of spikes (in t/stepSize)
        updateSpikeTime(t_ind)
        #Updates the input synaptic current to be used in RK4
        updateSyn(t_ind)
        #A function to update the solutions for all neurons' D.E.s
        RK4(t_ind)

        zeroTempVars() #Resets temporary variables like Isyn

    return 

mainProgramLoop()